In [2]:
import numpy as np
from tqdm import tqdm 
from sklearn.decomposition import PCA
from utils import get_distance_matrix, get_average_compression

In [3]:
def sample_spherical(npoints, ndim):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    vec *= np.random.randint(10, 500)

    return vec

In [4]:
def sample_normal(npoints, ndim=3, mu=0, sigma=1): 
    vec = np.random.normal(mu, sigma, (npoints, ndim))
    vec /= np.linalg.norm(vec, axis=0)
    return vec

In [5]:
# Number of clusters
k = 3
# Points per cluster
# n = 750
n = [1000, 750, 500]
# Number of dimensions
d = 100

In [6]:
# Generate cluster centers"
# x = sample_spherical(k, d).T
# x = np.empty((0, d))
# for i in range(k): 
#     x = np.vstack((x, sample_spherical(1, d).T))

alpha=10

x = alpha*np.zeros((1, d))
x = np.vstack((x, alpha*(np.zeros((1, d)) + 1)))
x = np.vstack((x, alpha*(np.zeros((1, d)) + 0.5)))
x  
           
# p = 0.5
# x = np.empty((0, d))
# x = np.vstack((x, np.random.choice([1, 0, -1], size=(len(n), d), p=[p, (1-p)/2., (1-p)/2.])))
# x

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10

In [7]:
# Compute pairwise distances
distances = np.zeros((k, k))
for i in tqdm(range(k)):
    for j in range(k):
        distances[i,j] = np.linalg.norm(x[i]-x[j])

100%|██████████| 3/3 [00:00<00:00, 8019.70it/s]


In [8]:
distances

array([[  0., 100.,  50.],
       [100.,   0.,  50.],
       [ 50.,  50.,   0.]])

In [231]:
# Add outlier as average of cluster centers
x = np.vstack((x, np.mean(x, axis=0)))

# Add outlier as weighted average of cluster centers
# weights = [0.1, 0.5, 0.2, 0.2]
# x = np.vstack((x, np.average(x, axis=0, weights=weights)))

# Add outlier as random point
# x = np.vstack((x, sample_spherical(1, d).T))

In [232]:
# Compute pairwise distances
distancesOut = np.zeros((k+1, k+1))
for i in tqdm(range(k+1)):
    for j in range(k+1):
        distancesOut[i,j] = np.linalg.norm(x[i]-x[j])

100%|██████████| 4/4 [00:00<00:00, 6450.29it/s]


In [233]:
distancesOut

array([[  0., 100.,  50.,  50.],
       [100.,   0.,  50.,  50.],
       [ 50.,  50.,   0.,   0.],
       [ 50.,  50.,   0.,   0.]])

In [9]:
# Create data Y = X + E
sigma = 2
Y = np.empty((0, d))
p = 0.3
for i in range(k): 
    for j in range(n[i]):
        # Y = np.vstack((Y, x[i] + np.random.normal(0, sigma, d)))
        point = x[i] + np.random.choice([50, 0, -50], size=(1, d), p=[(1-p)/2., p, (1-p)/2.])
        Y = np.vstack((Y, point))

In [10]:
Y[100:]

array([[ 50., -50.,   0., ...,  50., -50., -50.],
       [  0.,   0., -50., ...,  50., -50.,  50.],
       [  0.,  50.,  50., ..., -50.,  50.,  50.],
       ...,
       [  5.,  55., -45., ..., -45.,  55.,   5.],
       [ 55.,   5., -45., ...,  55.,  55.,   5.],
       [ 55.,   5.,  55., ...,  55., -45.,   5.]])

In [188]:
avg_intracluster_dist = np.zeros(k)
avg_intercluster_dist = np.zeros(k)
for i in range(k):
    before = sum(n[:i])

    for j in range(before, before + n[i]): 
        for l in range(before, before + n[i]):
            if j == l: 
                continue 
            avg_intracluster_dist[i] += np.linalg.norm(Y[j] - Y[l])
    
        t1 = range(before, before + n[i])
        t2 = range(Y.shape[0])
        t3 = set(t2).difference(set(t1))

        for l in list(t3):
            avg_intercluster_dist[i] += np.linalg.norm(Y[j] - Y[l])

for i in range(k):
    avg_intracluster_dist[i] /= (n[i] * (n[i] - 1))
    avg_intercluster_dist[i] /= ((sum(n) - n[i]) * n[i])

print(avg_intracluster_dist)
print(avg_intercluster_dist)

[11.80830784 11.80127469 11.7873195 ]
[620.49609544 983.30775832 436.46849993]


In [11]:
# Add small number of outliers based on outlier center
# on = 10
# for i in range(on): 
#     Y = np.vstack((Y, x[k] + np.random.normal(0, sigma, d)))

# Outliers on the surface of a sphere
on = 10
# outliers = sample_spherical(on, d).T
for i in range(on): 
    outlier = sample_spherical(1, d).T
    Y = np.vstack((Y, outlier))


In [12]:
l2_points = [Y[0], Y[1000], Y[1750]]
for i in range(2250, 2260): 
    l2_points.append(Y[i])

In [13]:
l2_points = np.array(l2_points)
dist_l2 = np.linalg.norm(l2_points)

for i in l2_points: 
    print(np.linalg.norm(i))

# for i in range(len(l2_points)): 
#     for j in range(len(l2_points)):
#         dist_l2[i,j] = np.linalg.norm(l2_points[i]-l2_points[j])

# print(dist_l2)

441.5880433163923
435.3159771935783
406.81691213615983
94.99999999999999
423.0
377.0
391.0
378.0
415.00000000000006
233.00000000000003
370.0
232.99999999999997
342.0


In [14]:
totalPoints = sum(n) + 10

In [15]:
Y

array([[-50.        , -50.        ,  50.        , ...,  50.        ,
         50.        ,  50.        ],
       [ 50.        ,  50.        , -50.        , ...,  50.        ,
          0.        , -50.        ],
       [  0.        , -50.        ,  50.        , ...,   0.        ,
         50.        ,   0.        ],
       ...,
       [-20.81176583,  26.25087269, -26.50035465, ..., -51.91946792,
         92.63778905, -54.14809399],
       [ 24.08927564,  -0.99143165, -20.04614041, ...,  13.94971719,
          7.036582  ,   3.31127339],
       [-22.50847363, -29.73801907,  13.09731475, ...,   7.79782664,
          2.1845227 ,  17.78361466]])

In [16]:
len(Y)

2260

In [17]:
# Compute Euclidean distance matrix between each point
D_pre = np.zeros((totalPoints, totalPoints))
for i in tqdm(range(totalPoints)):
    for j in range(totalPoints):
        D_pre[i, j] = np.linalg.norm(Y[i] - Y[j])

100%|██████████| 2260/2260 [00:08<00:00, 252.17it/s]


In [18]:
D_pre_util = get_distance_matrix(Y)

In [19]:
D_pre

array([[  0.        , 634.42887702, 618.46584384, ..., 584.54075417,
        488.5853348 , 507.4085646 ],
       [634.42887702,   0.        , 648.07406984, ..., 563.24130035,
        463.32430575, 509.49804755],
       [618.46584384, 648.07406984,   0.        , ..., 532.51006829,
        512.63484286, 572.16661612],
       ...,
       [584.54075417, 563.24130035, 532.51006829, ...,   0.        ,
        440.17897132, 508.41543754],
       [488.5853348 , 463.32430575, 512.63484286, ..., 440.17897132,
          0.        , 434.46791608],
       [507.4085646 , 509.49804755, 572.16661612, ..., 508.41543754,
        434.46791608,   0.        ]])

In [20]:
components = 4
pca = PCA(n_components=components)
pca.fit(Y)
Y_pca = pca.transform(Y)

In [21]:
Y_pca

array([[ -31.31237037, -101.54932745,  -40.35616506,   -4.12333116],
       [   0.35136712,   42.86121058,    4.21193765,   31.85360846],
       [ -45.26922548,  -46.03757893,   -9.64883754,  -46.03294158],
       ...,
       [ -16.79097908,   40.52618621,   -1.93877869,  -27.89523446],
       [ -85.30901283,  -27.78901772,  -16.43244023,  -49.27361851],
       [ -19.20422876,   -4.97510651,    6.56568201,   22.28077271]])

In [23]:
# Compute Euclidean distance matrix between each point post PCA
D_post = np.zeros((totalPoints, totalPoints))
for i in tqdm(range(totalPoints)):
    for j in range(totalPoints):
        D_post[i, j] = np.linalg.norm(Y_pca[i] - Y_pca[j])

100%|██████████| 2260/2260 [00:11<00:00, 198.57it/s]


In [22]:
D_post = get_distance_matrix(Y_pca)

In [23]:
D_post

array([[  0.        , 158.54857846,  77.3026742 , ..., 149.79092546,
        104.7239884 , 111.22959893],
       [158.54857846,   0.        , 127.44673009, ...,  62.50654297,
        139.05744405,  52.6109671 ],
       [ 77.3026742 , 127.44673009,   0.        , ...,  93.23474341,
         44.63982313,  85.41185456],
       ...,
       [149.79092546,  62.50654297,  93.23474341, ...,   0.        ,
        100.14384437,  68.30921578],
       [104.7239884 , 139.05744405,  44.63982313, ..., 100.14384437,
          0.        , 102.66090566],
       [111.22959893,  52.6109671 ,  85.41185456, ...,  68.30921578,
        102.66090566,   0.        ]])

In [27]:
D_post_util

array([[ 0.        , 55.25853077, 76.17191496, ..., 46.1558036 ,
        66.72340041, 50.42629763],
       [55.25853077,  0.        , 97.09402597, ..., 34.91195272,
        75.38144397, 31.62205541],
       [76.17191496, 97.09402597,  0.        , ..., 69.89925833,
        56.82166229, 75.65488787],
       ...,
       [46.1558036 , 34.91195272, 69.89925833, ...,  0.        ,
        52.50908131,  7.35317195],
       [66.72340041, 75.38144397, 56.82166229, ..., 52.50908131,
         0.        , 53.50070376],
       [50.42629763, 31.62205541, 75.65488787, ...,  7.35317195,
        53.50070376,  0.        ]])

In [24]:
C = D_pre / D_post

/var/folders/p0/nb5t36rs6p5_2b9yzyw6y8hw0000gn/T/ipykernel_4831/2875983867.py:1: RuntimeWarning: invalid value encountered in divide
  C = D_pre / D_post


In [25]:
C

array([[        nan,  4.00147944,  8.00057502, ...,  3.90237761,
         4.66545767,  4.56181241],
       [ 4.00147944,         nan,  5.08505843, ...,  9.01091748,
         3.33189143,  9.6842555 ],
       [ 8.00057502,  5.08505843,         nan, ...,  5.71149819,
        11.48380094,  6.69891339],
       ...,
       [ 3.90237761,  9.01091748,  5.71149819, ...,         nan,
         4.39546708,  7.44285279],
       [ 4.66545767,  3.33189143, 11.48380094, ...,  4.39546708,
                nan,  4.23206783],
       [ 4.56181241,  9.6842555 ,  6.69891339, ...,  7.44285279,
         4.23206783,         nan]])

In [26]:
cluster_sizes = n + [on]
avg_intracluster_compression = np.zeros(k + 1)
avg_intercluster_compression = np.zeros(k + 1)
for i in range(k + 1):
    before = sum(cluster_sizes[:i])

    for j in range(before, before + cluster_sizes[i]): 
        for l in range(before, before + cluster_sizes[i]):
            if j == l: 
                continue 
            avg_intracluster_compression[i] += C[j, l]
    
        t1 = range(before, before + cluster_sizes[i])
        t2 = range(Y.shape[0])
        t3 = set(t2).difference(set(t1))

        for l in list(t3):
            avg_intercluster_compression[i] += C[j, l]

for i in range(k + 1):
    avg_intracluster_compression[i] /= ((cluster_sizes[i] * (cluster_sizes[i] - 1)) / 2)
    avg_intercluster_compression[i] /= (cluster_sizes[i] * (len(Y) - cluster_sizes[i]))

In [27]:
avg_intercluster_compression

array([4.58582729, 4.51601432, 5.10852707, 5.24351963])

In [28]:
avg_intracluster_compression

array([10.7957197 , 10.67976928, 11.1471021 , 14.65613869])

In [29]:
print(cluster_sizes)

[1000, 750, 500, 10]


In [50]:
from utils import get_average_compression

In [58]:
def gac(c, cs, a): 
    """
    Compute the average intracluster and intercluster compression for a set of points.
    :param C: numpy array of pairwise compression ratios (n_samples, n_samples)
    :param cluster_sizes: array of cluster sizes
    :param k: number of clusters (including outlier "cluster")
    """
    aintac = np.zeros(a)
    aintec = np.zeros(a)
    for i in range(a):
        before = sum(cs[:i])

        for j in range(before, before + cs[i]): 
            for l in range(before, before + cs[i]):
                if j == l: 
                    continue 
                aintac[i] += c[j, l]
        
            t1 = range(before, before + cs[i])
            t2 = range(c.shape[0])
            t3 = set(t2).difference(set(t1))

            for l in list(t3):
                aintec[i] += c[j, l]

    for i in range(a):
        aintac[i] /= ((cs[i] * (cs[i] - 1)) / 2)
        aintec[i] /= (cs[i] * (len(C) - cs[i]))
    
    return aintec, aintac

In [63]:
from utils import get_average_compression

In [30]:
avg_inter_util, avg_intra_util = get_average_compression(C, cluster_sizes, k + 1)

In [31]:
avg_inter_util

array([4.58582729, 4.51601432, 5.10852707, 5.24351963])

In [ ]:
## TODO: 
# 1. Add outliers to single cell datasets, ex. n = 2000, 20 outliers. Show that outlier compressibility is very bad compared to this clusters
#### - Modify centers, noise to create artificial outliers with bad compressibility compared to clusters 
#### - Do the same thing with single cell



# For each outlier point, randomly sample point from each cluster, and compute average 